In [1]:
import torch
import numpy as np
import torchvision.datasets

In [40]:
# https://github.com/oval-group/dfw
from dfw import DFW
from dfw.losses import MultiClassHingeLoss

In [4]:
import torch.utils.data

In [5]:
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor 
image_data_train = torchvision.datasets.MNIST(root='./',train=True,download=True,transform=ToTensor())
image_data_test = torchvision.datasets.MNIST(root='./',train=False,download=True,transform=ToTensor())

In [67]:
data_loader_train = torch.utils.data.DataLoader(image_data_train,
                                          batch_size=8,
                                          shuffle=True,
                                          )

data_loader_test = torch.utils.data.DataLoader(image_data_test,
                                          batch_size=8,
                                          shuffle=True,
                                          )

In [73]:
import torch.nn as nn
from torch.autograd import Function

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

inpsiz = 784 
hidensiz = 512 
numclases = 10
numepchs = 4
bachsiz = 100
l_r = 0.001

class neural_network(nn.Module):
    def __init__(self, inpsiz, hidensiz, numclases):
        super(neural_network, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
modl = neural_network(inpsiz, hidensiz, numclases)
loss = MultiClassHingeLoss()
# create DFW optimizer with learning rate of 0.1
optim = DFW(modl.parameters(), eta=0.1)
# train(data_loader_train,modl,loss,optim)
print(modl)

Using cpu device
neural_network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [69]:
data_loader_train.batch_size

8

In [74]:
## training on batches
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    loss_list=[]
    for batch, (X, y) in enumerate(dataloader):
        # one X is a batch of 8 instances
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_list.append(loss)
    # Backpropagation of all batches together
    optimizer.zero_grad()
    for loss in loss_list:
        loss.backward()
    optimizer.step(lambda: float(loss))


    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



In [72]:
# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     model.train()
#     pred
#     for batch, (X, y) in enumerate(dataloader):
#         # one X is a batch of 8 instances
#         X, y = X.to(device), y.to(device)

#         # Compute prediction error
#         pred = model(X)
#         loss = loss_fn(pred, y)
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step(lambda: float(loss))

#         if batch % 100 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# loss = MultiClassHingeLoss()
# # create DFW optimizer with learning rate of 0.1
# optim = DFW(modl.parameters(), eta=0.1)
# # train(data_loader_train,modl,loss,optim)

In [54]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [79]:
pwd

'C:\\Users\\Akshit Bhatia\\OneDrive - Blue Avenir GmbH\\sebestian thesis\\torch_mnist\\torch_mnist'

In [75]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(data_loader_train, modl, loss, optim)
#     test(data_loader_test, modl, loss)
print("Done!")

Epoch 1
-------------------------------
loss: 1.074328  [59992/60000]
Epoch 2
-------------------------------
loss: 1.061776  [59992/60000]
Epoch 3
-------------------------------
loss: 1.066381  [59992/60000]
Epoch 4
-------------------------------
loss: 1.092778  [59992/60000]
Epoch 5
-------------------------------
loss: 1.072833  [59992/60000]
Done!


In [76]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
#     train(data_loader_train, modl, loss, optim)
    test(data_loader_test, modl, loss)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 9.8%, Avg loss: 1.071823 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 9.8%, Avg loss: 1.071823 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 9.8%, Avg loss: 1.071823 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 9.8%, Avg loss: 1.071823 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 9.8%, Avg loss: 1.071823 

Done!


In [15]:

nttlstps = len(data_loader_train)

for epoch in range(10):
    for bachsiz, (imgs,lbls) in enumerate(data_loader_train):
        imgs=imgs.reshape(-1,28*28)
        labls=lbls
        
        outp=modl(imgs)
        losses=loss(outp,lbls)
        optimizer.zero_grad()
        losses.backward()
        optimizer.step(lambda: float(losses))
    if((x+1)%100 == 0):
    
        print (f'Epochs [{epoch+1}/{numepchs}], Step[{x+1}/{nttlstps}], Losses: {losses.item():.4f}')


# # DFW can be used with standard pytorch syntax
# optimizer.zero_grad()
# loss = svm(model(x), y)
# loss.backward()
# # NB: DFW needs to have access to the current loss value,
# # (this syntax is compatible with standard pytorch optimizers too)
# optimizer.step(lambda: float(loss))

Epochs [1/4], Step[15000/15000], Losses: 0.4618
Epochs [2/4], Step[15000/15000], Losses: 0.0000
Epochs [3/4], Step[15000/15000], Losses: 0.0000
Epochs [4/4], Step[15000/15000], Losses: 0.0000
Epochs [5/4], Step[15000/15000], Losses: 0.0000
Epochs [6/4], Step[15000/15000], Losses: 0.0000
Epochs [7/4], Step[15000/15000], Losses: 0.0000
Epochs [8/4], Step[15000/15000], Losses: 0.1499
Epochs [9/4], Step[15000/15000], Losses: 0.0000
Epochs [10/4], Step[15000/15000], Losses: 0.0000


In [20]:
def accuracy(out, targets, topk=1):
    if topk == 1:
        _, pred = torch.max(out, 1)
        acc = torch.mean(torch.eq(pred, targets).float())
    else:
        _, pred = out.topk(topk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(targets.view(1, -1).expand_as(pred))
        acc = correct[:topk].view(-1).float().sum(0) / out.size(0)

    return 100. * acc

In [23]:
for x, (imgs,lbls) in enumerate(data_loader_test):
        imgs=imgs.reshape(-1,28*28)
        labls=lbls
        
        outp=modl(imgs)
#         print(accuracy(outp,labls))
        print(loss(outp,labls))

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.4742, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.4302, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1983, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2858, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tens

tensor(0.0269, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.4473, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0499, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0610, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2477, grad_fn=<DivBackward0>)
tensor(0.2753, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0006, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBack

tensor(0.0218, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1782, grad_fn=<DivBackward0>)
tensor(0.1320, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(1.0246, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0364, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.4454, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0225, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1144, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<Div

tensor(0.3261, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1187, grad_fn=<DivBackward0>)
tensor(0.1970, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2321, grad_fn=<DivBackward0>)
tensor(0.0986, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0960, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.9822, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBack

tensor(0.0079, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0292, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.3005, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0364, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.3967, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1627, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.4128, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.5002, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.3597, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2327, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2074, grad_fn=<DivBackward0>)


tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2843, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1199, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0210, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0382, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1696, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)


tensor(0.1986, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0615, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0296, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0492, grad_fn=<DivBackward0>)
tens

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0287, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2517, grad_fn=<DivBackward0>)
tensor(0.5176, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.4134, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tens

In [22]:
for x, (imgs,lbls) in enumerate(data_loader_test):
        imgs=imgs.reshape(-1,28*28)
        labls=lbls
        
        outp=modl(imgs)
#         print(accuracy(outp,labls))
#         print(loss(outp,labls))
        

tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(75.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(50.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(1

tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)


tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tens

tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(75.)
tensor(100.)
tensor(100.)
t

In [24]:
criter = nn.CrossEntropyLoss()
optim = torch.optim.Adam(modl.parameters(), lr=l_r)
nttlstps = len(data_loader_train)

for epoch in range(10):
    for x, (imgs, lbls) in enumerate(data_loader_train): 
         imgs = imgs.reshape(-1, 28*28)
         labls = lbls

         outp = modl(imgs)
         losses = criter(outp, lbls)

         optim.zero_grad()
         losses.backward()
         optim.step() 
    if (x+1) % 100 == 0:
             print (f'Epochs [{epoch+1}/{numepchs}], Step[{x+1}/{nttlstps}], Losses: {losses.item():.4f}')

Epochs [1/4], Step[15000/15000], Losses: 0.0001
Epochs [2/4], Step[15000/15000], Losses: 0.4092
Epochs [3/4], Step[15000/15000], Losses: 0.0035
Epochs [4/4], Step[15000/15000], Losses: 0.0000
Epochs [5/4], Step[15000/15000], Losses: 0.0000
Epochs [6/4], Step[15000/15000], Losses: 0.0000
Epochs [7/4], Step[15000/15000], Losses: 0.0001
Epochs [8/4], Step[15000/15000], Losses: 0.0000
Epochs [9/4], Step[15000/15000], Losses: 0.0000
Epochs [10/4], Step[15000/15000], Losses: 0.0000


In [25]:
for x, (imgs,lbls) in enumerate(data_loader_test):
        imgs=imgs.reshape(-1,28*28)
        labls=lbls
        
        outp=modl(imgs)
#         print(accuracy(outp,labls))
        print(loss(outp,labls))

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.9361, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.9105, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., g

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.3870, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0221, grad_fn=<DivBackward0>)
tensor(2.4437, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.1856, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.9348, grad_fn=<DivBackward0>)


tensor(0., grad_fn=<DivBackward0>)
tensor(0.5190, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(1.1025, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(1.5665, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.3102, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(1.5428, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.2464, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(1.2487, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.9762, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(1.4298, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(2.8807, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.5540, grad_fn=<DivBackward0>)
tensor(0.6886, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tens

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(3.0786, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(2.6251, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., g

tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(5.0900, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(1.3737, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0009, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0

In [28]:
results_l=[]
for x, (imgs,lbls) in enumerate(data_loader_test):
        imgs=imgs.reshape(-1,28*28)
        labls=lbls
        
        outp=modl(imgs)
        results_l.append(accuracy(outp,labls))
#         print(loss(outp,labls))
        

In [36]:
dir(results_l[0])

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed_

In [39]:
results_l[0]==tensor(100.)

NameError: name 'tensor' is not defined